# PSScene Transition Workshop
This workshop is meant to outline the changes that you will need to implement during the PSScene migration. We will cover the Data API, Order's API and Subscription API.

Quick links:

[PSScene3Band and PSScene4Band depretiation document](https://developers.planet.com/docs/apis/data/psscene3-4band-deprecation/)

[PSScene Documentation](https://developers.planet.com/docs/data/psscene/)

All other notebooks on the [Planet Github](https://github.com/planetlabs/notebooks/tree/master/jupyter-notebooks) already reflect these changes already.


In [ ]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth

# API Key stored as an env variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

## Data API
Here is an outline of a request body to make a quicksearch. The comments highlight the areas that will need changing during the migration.

In [ ]:
# Stockton, CA bounding box (created via geojson.io) 
geojson_geometry = {
  "type" : "Polygon", 
  "coordinates": [[[
              -122.56587982177734,
              37.69088430259205
            ],
            [
              -122.3444366455078,
              37.69088430259205
            ],
            [
              -122.3444366455078,
              37.814123701604466
            ],
            [
              -122.56587982177734,
              37.814123701604466
            ],
            [
              -122.56587982177734,
              37.69088430259205
            ]]]
}

In [ ]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gt":"2021-12-31T00:00:00Z",
    "lte":"2022-01-31T00:00:00Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

Now switch it to ```PSScene``` and change the `asset_filter`. All of the assets will either start with ortho or basic.

In [ ]:
item_type = "PSScene4Band" #PSScene


asset_filter = {
        "type": "AssetFilter",
        "config": ["analytic_sr"] #ortho_analytic_4b_sr
      }

combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter, asset_filter]
}

In [ ]:

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)


print(json.dumps(search_result.json(), indent=1)) 

In [ ]:
# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
print(image_ids)

## Order's API
This section takes an ID from above and orders that scene with the correct inputs. Followed by scripts to download them. Same as above the new PSScene changes are commented in. 

In [ ]:
orders_url = 'https://api.planet.com/compute/ops/orders/v2'
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

# set content type to json
headers = {'content-type': 'application/json'}

Change `item_type` and ```product_bundles```

In [ ]:
request = {  
   "name":"PSScene order",
   "products":[
      {  
         "item_ids":[  
            '20220130_183458_12_249d'
         ],
         "item_type":"PSScene4band", #PSScene
         "product_bundle":"analytic" #analytic_udm2
      }
   ],
}

In [ ]:
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response.json())
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

You can see the `item_type` and ```product_bundles``` in the response

In [ ]:
order_url = place_order(request, auth)

The tool supports 15 band calculations so customers can get all the raw bands of an 8-band asset and perform 7 calculations

In [ ]:
bandmath = {
  "bandmath": {
    "pixel_type": "32R",
    "b1": "b1",
    "b2": "b2",
    "b3": "b3",
    "b4": "b4",
    "b5": "b5",
    "b6": "b6",
    "b7": "b7",
    "b8": "b8",
    "b9": "(b8 - b6) / (b8 + b6)", #NDVI
    "b10": "(b4 - b8) / (b4 + b6)" #NDWI
  }
}

In [ ]:
bandmath_request = {  
   "name":"bandmath order",
   "products":[
      {  
         "item_ids":[  
            '20220130_183458_12_249d'
         ],
         "item_type":"PSScene4Band", #PSScene
         "product_bundle":"analytic_udm2" #analytic_udm2
      }
   ],
    "tools": [bandmath]
}

bandmath_order_url = place_order(bandmath_request, auth)

In [ ]:
import pathlib
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

In [ ]:
requests.get(order_url, auth=auth).json()['state']

Simply changing to an old order for the demo as the other one is still running

In [ ]:
#old_order_url = "https://api.planet.com/compute/ops/orders/v2/1726717d-54b8-4601-bd62-5108b481f69f"
r = requests.get(old_order_url, auth=auth)
response = r.json()
results = response['_links']['results']

Be mindful that the folder naming has changed for `PSScene3band` and `PSScene4band` to `PSScene`

In [ ]:
[r['name'] for r in results]

In [ ]:
download_results(results)

## Subscription's API
Lastly this section shows the request body and how to create a subcription. In this case changing from the 3band visual asset to the PSScene visual asset.

In [ ]:
# set content type to json
headers = {'content-type': 'application/json'}

# set your delivery details
BUCKET_NAME = 'name of bucket'
GCS_CREDENTIALS= "credentials string"

In [ ]:
request = {
    "name": "Recurring Subscription",
    "source": {
        "type": "catalog",
        "parameters": {
            "geometry": {
                "coordinates": [[[139.5648193359375,35.42374884923695],
                                [140.1031494140625,35.42374884923695],
                                [140.1031494140625,35.77102915686019],
                                [139.5648193359375,35.77102915686019],
                                [139.5648193359375,35.42374884923695]]],
                "type": "Polygon"
            },
            "start_time": "2022-03-01T00:00:00Z",
            "end_time": "2023-11-01T00:00:00Z",
            "rrule": "FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10",
            "item_types": ["PSScene3band"],  #PSScene
            "asset_types": ["visual"] #ortho_visual
        }
    },
    "delivery": { 
        "type": "google_cloud_storage",
        "parameters": {
            "bucket": BUCKET_NAME,
            "credentials": GCS_CREDENTIALS,
                
        }
    }
}

In [ ]:
def place_subscription(request, auth):
    response = requests.post(subscriptions_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    subscriptions_id = response.json()['id']
    print(subscriptions_id)
    subscription_url = subscriptions_url + '/' + subscriptions_id
    return subscription_url